In [ ]:
import networkx as nx
import numpy as np

In [ ]:
def create_base_graph():
    """
    Create the base directed graph with edges (no probabilities yet).
    """
    G = nx.DiGraph()
    
    edges = [
        (1, 5), (5, 15), (15, 12), (11, 13), (15, 13),
        (3, 6), (3, 8), (6, 8), (4, 7), (2, 9),
        (2, 10), (8, 10), (7, 10), (2, 11), (10, 15),
        (8, 14), (9, 14), (11, 14), (7, 14), (11, 16),
        (7, 16), (2, 16), (8, 16), (15, 16)
    ]
    
    G.add_edges_from(edges)
    return G

def add_edge_probabilities(G):
    """
    Add 'prob' attribute to edges, matching the exact R internal ordering.
    """
    edges_in_order = [
        (1, 5),
        (5, 15),
        (15, 12),
        (15, 13),
        (15, 16),
        (11, 13),
        (11, 14),
        (11, 16),
        (3, 6),
        (3, 8),
        (6, 8),
        (8, 10),
        (8, 14),
        (8, 16),
        (4, 7),
        (7, 10),
        (7, 14),
        (7, 16),
        (2, 11),
        (2, 9),
        (2, 10),
        (2, 16),
        (9, 14),
        (10, 15)
    ]
    
    edge_probs = [
        0.111265,
        0.111265,
        0.472876,
        0.472876,
        0.472876,
        0.344921,
        0.472876,
        1.000000,
        0.344921,
        0.472876,
        1.000000,
        1.000000,
        1.000000,
        0.472876,
        0.472876,
        0.472876,
        0.472876,
        0.472876,
        0.472876,
        0.472876,
        0.344921,
        0.344921,
        0.344921,
        1.000000
    ]
    
    for (u, v), p in zip(edges_in_order, edge_probs):
        if G.has_edge(u, v):
            G[u][v]['prob'] = p
        else:
            raise ValueError(f"Edge ({u}->{v}) not found in the graph. Check definitions!")
    
    return G


def add_edge_weights(G):
    """
    'weight' = -log(prob), same as the R code (with potential -0.0 -> 0.0 cleanup).
    """
    for u, v in G.edges():
        p = G[u][v]['prob']
        w = -np.log(p)
        if abs(w) < 1e-14:
            w = 0.0
        G[u][v]['weight'] = w

    return G

In [12]:
def create_mir100_attack_graph():
    """
    Create the graph, add probabilities, compute weights, etc.
    """
    G = create_base_graph()
    G = add_edge_probabilities(G)
    G = add_edge_weights(G)
    return G

attack_graph = create_mir100_attack_graph()

In [ ]:
def visualize_mir100_graph(attack_graph):
    """
    Visualizes the MIR100 attack graph with the same styling as the visualize_graphs function.
    """
    # Create subplot figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    original_target_nodes = [n for n, d in attack_graph.out_degree() if d == 0]
    
    entry_nodes = [n for n, d in attack_graph.in_degree() if d == 0 and n not in original_target_nodes]

    pos = nx.spring_layout(attack_graph, k=1, iterations=50, seed=42)
    
    nx.draw_networkx_edges(attack_graph, pos, 
                          edge_color='gray',
                          arrows=True,
                          arrowsize=15,
                          width=1.5,
                          ax=ax)
    
    node_colors = []
    for node in attack_graph.nodes():
        if node in original_target_nodes:
            node_colors.append('lightcoral')  
        elif node in entry_nodes:
            node_colors.append('lightgreen') 
        else:
            node_colors.append('lightblue')   
    
    nx.draw_networkx_nodes(attack_graph, pos,
                          node_color=node_colors,
                          node_size=500,
                          edgecolors='darkblue',
                          linewidths=1.5,
                          ax=ax)
    
    labels = {node: str(node) for node in attack_graph.nodes()}
    nx.draw_networkx_labels(attack_graph, pos,
                           labels,
                           font_size=10,
                           font_weight='bold',
                           ax=ax)
    
    ax.set_title("MIR100 Attack Graph", fontsize=12, fontweight='bold')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
if image_mode:
    visualize_mir100_graph(attack_graph)